# 城市推荐系统模型训练

本Notebook用于训练城市推荐系统的模型，包括数据预处理、聚类分析和回归模型训练等步骤。

## 1. 导入必要的库

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
import numpy as np
import os

# 设置并行线程数，避免内存溢出
os.environ['OMP_NUM_THREADS'] = '1'

## 2. 数据加载函数

该函数用于加载城市数据集，并进行基本的错误处理。

In [2]:
def load_data():
    try:
        # 加载数据
        df = pd.read_csv('updated_city_happiness.csv', encoding='utf-8-sig')
        return df
    except FileNotFoundError:
        print('数据文件未找到，请检查文件路径和文件名。')
        return None

## 3. 数据预处理函数

该函数用于提取特征并进行标准化处理，为聚类分析做准备。

In [3]:
def preprocess_data(df):
    # 提取特征
    features = df[['人均可支配收入(万元)', '房价收入比', '教育满意度(10分制)', '医疗资源指数', 
                   'PM2.5年均值', '公园绿地面积(㎡/人)', '养老保险覆盖率(%)', '每万人警力数', 
                   '通勤时间(分钟)']]

    # 对特征进行标准化
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    return features_scaled, scaler

## 4. 寻找最佳聚类数函数

该函数使用轮廓系数法寻找最佳的聚类数量，提高聚类效果。

In [4]:
def find_best_k(features_scaled):
    # 使用轮廓系数法选择最佳的簇数
    silhouette_scores = []
    for k in range(2, 11):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(features_scaled)
        labels = kmeans.labels_
        score = silhouette_score(features_scaled, labels)
        silhouette_scores.append(score)

    # 获取最佳簇数
    best_k = silhouette_scores.index(max(silhouette_scores)) + 2
    return best_k

## 5. 执行聚类函数

该函数使用最佳聚类数对城市数据进行聚类，并将聚类结果添加到数据集中。

In [5]:
def perform_clustering(df, features_scaled, best_k):
    # 使用最佳簇数进行K-means聚类
    kmeans = KMeans(n_clusters=best_k, random_state=42)
    df['Cluster'] = kmeans.fit_predict(features_scaled)
    return kmeans

## 6. 构建并保存模型函数

该函数为每个聚类构建随机森林回归模型，用于预测幸福指数，并保存模型。

In [6]:
def build_and_save_models(df):
    # 为每个聚类构建随机森林回归模型并预测幸福指数
    models = {}
    for cluster in df['Cluster'].unique():
        cluster_data = df[df['Cluster'] == cluster]
        X = cluster_data[['人均可支配收入(万元)', '房价收入比', '教育满意度(10分制)', '医疗资源指数', 
                          'PM2.5年均值', '公园绿地面积(㎡/人)', '养老保险覆盖率(%)', '每万人警力数', 
                          '通勤时间(分钟)']]
        y = cluster_data['幸福指数']

        # 划分训练集和测试集
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # 构建随机森林回归模型
        rf = RandomForestRegressor(random_state=42)
        rf.fit(X_train, y_train)

        # 保存模型
        models[cluster] = rf

    # 保存随机森林模型字典
    joblib.dump(models, 'suijisenlinzidian.pkl')
    return models

## 7. 主函数

该函数整合了上述所有步骤，完成从数据加载到模型训练和保存的全过程。

In [7]:
def main():
    df = load_data()
    if df is not None:
        print(f'成功加载数据，共{len(df)}条记录')

        # 数据预处理
        print('正在进行数据预处理...')
        features_scaled, scaler = preprocess_data(df)

        # 寻找最佳聚类数
        print('正在寻找最佳聚类数...')
        best_k = find_best_k(features_scaled)
        print(f'最佳聚类数为: {best_k}')

        # 执行聚类
        print('正在执行聚类分析...')
        kmeans = perform_clustering(df, features_scaled, best_k)

        # 保存K-means模型和缩放器
        joblib.dump(kmeans, 'kmeans_model.pkl')
        joblib.dump(scaler, 'TeZhengBiaoZhunHua.pkl')

        # 保存带有Cluster列的数据集
        print('正在保存处理后的数据集...')
        df.to_csv('updated_city_happiness_with_cluster.csv', index=False)

        # 构建并保存回归模型
        print('正在构建回归模型...')
        build_and_save_models(df)

        print('模型训练和数据处理完成，已保存带有Cluster列的数据集')
        print('生成的文件包括:')
        print('- kmeans_model.pkl: K-means聚类模型')
        print('- TeZhengBiaoZhunHua.pkl: 特征标准化模型')
        print('- suijisenlinzidian.pkl: 随机森林回归模型字典')
        print('- updated_city_happiness_with_cluster.csv: 带有聚类结果的数据集')

if __name__ == "__main__":
    main()

成功加载数据，共221条记录
正在进行数据预处理...
正在寻找最佳聚类数...
最佳聚类数为: 2
正在执行聚类分析...


c:\Users\15339\.conda\envs\datascience\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\15339\.conda\envs\datascience\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\15339\.conda\envs\datascience\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\15339\.conda\envs\datascience\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: User

正在保存处理后的数据集...
正在构建回归模型...
模型训练和数据处理完成，已保存带有Cluster列的数据集
生成的文件包括:
- kmeans_model.pkl: K-means聚类模型
- TeZhengBiaoZhunHua.pkl: 特征标准化模型
- suijisenlinzidian.pkl: 随机森林回归模型字典
- updated_city_happiness_with_cluster.csv: 带有聚类结果的数据集


## 8. 运行模型训练

执行以下代码块，开始模型训练过程。

In [8]:
# 运行模型训练
main()

成功加载数据，共221条记录
正在进行数据预处理...
正在寻找最佳聚类数...
最佳聚类数为: 2
正在执行聚类分析...
正在保存处理后的数据集...
正在构建回归模型...


c:\Users\15339\.conda\envs\datascience\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\15339\.conda\envs\datascience\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\15339\.conda\envs\datascience\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\15339\.conda\envs\datascience\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: User

模型训练和数据处理完成，已保存带有Cluster列的数据集
生成的文件包括:
- kmeans_model.pkl: K-means聚类模型
- TeZhengBiaoZhunHua.pkl: 特征标准化模型
- suijisenlinzidian.pkl: 随机森林回归模型字典
- updated_city_happiness_with_cluster.csv: 带有聚类结果的数据集
